In [1]:
import numpy as np
import pandas as pd

In [46]:
pros = pd.read_csv('/Users/clairefenton/Downloads/2023-01-25_name_removed/Prosecutor Data-01-25-2023_name_rem.csv')
sent = pd.read_csv('/Users/clairefenton/Downloads/2023-01-25_name_removed/Sentencing Information-01-25-2023_name_rem.csv')

In [47]:
len(sent)

90084

In [64]:
statutes = ['16-7-20', '16-9-1(B)','16-9-1(D)', '16-13-30(c)(1)', '16-9-31', '16-13-30(b)', '16-7-1(c)', '16-9-33', '16-8-18', '16-8-2', '16-8-14','16-7-1(b)', '16-7-1b', '16-9-121', '16-13-30', '16-8-7', '16-13-30(j)', '16-13-30(J)', '16-11-131(b)', '16-13-30(a)', '16-13-30(A)']
dispositions = ['Nolle Prosse', 'Nolle Prosse - Converted', 'Nolle Prosse - Restrict', 'Not Presented to Granted Jury - Restrict - Converted', 'Not Presented to Grand Jury', 'Not Presented to Grand Jury - Restrict']

sent = sent[(sent['Degree'] == 'Misdemeanor') | (sent['Statute'].isin(statutes))]

In [78]:
sent = sent.reset_index(drop=True)

In [ ]:
len(sent)
print(sent)

In [84]:
len(sent[sent['Degree'] == 'Felony'])/len(sent)

0.4492860494737548

In [81]:
policy_case = sent[['Statute', 'ChargeDisposition']]
flag = []
Treatment = []
print(len(policy_case))
iteration = list(range(0, len(policy_case)))

for i in iteration:
    if policy_case['Statute'][i] in statutes:
        flag.append(True)
        if policy_case['ChargeDisposition'][i] in dispositions:
            Treatment.append(True)
        else:
            Treatment.append(False)
    else:
        flag.append(False)
        Treatment.append(False)

print(len(flag))
print(len(Treatment))
print("Eligible Cases", sum(flag))
print("Percentage of eligible cases", sum(flag)/len(flag))
print("Policy Cases", sum(Treatment))
print("Percentage of policy-applied cases in eligible cases", sum(Treatment)/sum(flag))

44751
44751
44751
Eligible Cases 24421
Percentage of eligible cases 0.5457084757882505
Policy Cases 7725
Percentage of policy-applied cases in eligible cases 0.3163261127717948


In [85]:
sent['CaseFlag'] = flag
sent['PolicyApplied'] = Treatment

In [87]:
sent.to_csv('subsetSentencingData.csv')

In [60]:
pros.rename(columns={'CaseNbr':'CaseNumber'}, inplace=True)
pros.rename(columns={'UniquePersonID':'PartyID'}, inplace=True)
pros.columns

Index(['CaseType', 'Attorney', 'AttorneyAppointment', 'ProsAtt',
       'CaseOriginDate', 'JudgeandDivisionAssigned', 'PartyID', 'CaseNumber',
       'Race', 'Ethnicity', 'Gender', 'DOB', 'City', 'State', 'ZIP'],
      dtype='object')

In [7]:
race = pros.Race
race.value_counts()

B    28734
W     5310
O      444
A      311
I       53
H       50
U       43
S       20
M        2
Name: Race, dtype: int64

In [9]:
race.value_counts()/len(race)

B    0.818259
W    0.151213
O    0.012644
A    0.008856
I    0.001509
H    0.001424
U    0.001225
S    0.000570
M    0.000057
Name: Race, dtype: float64

In [ ]:
ethnicity = pros.Ethnicity
print(ethnicity.value_counts())
ethnicity.value_counts()/len(ethnicity)

In [ ]:
gender = pros.Gender
print(gender.value_counts())
gender.value_counts()/len(gender)

In [ ]:
prosna = pros[pros['City'].notna()]
prosna.head(10)

In [83]:
city = prosna.City
lower_cities = []

for name in city:
    lower_cities.append(name.lower())

lower_cities = pd.Series(lower_cities)
lower_cities.value_counts()[:10]/len(lower_cities)

atlanta           0.200679
decatur           0.198383
lithonia          0.098009
stone mountain    0.090333
clarkston         0.019259
ellenwood         0.019053
conyers           0.015661
norcross          0.015078
lawrenceville     0.014530
brookhaven        0.012645
dtype: float64

In [97]:
prosnazip = pros[pros['ZIP'].notna()]

In [100]:
zip = prosnazip.ZIP
zip.value_counts()

30032         2859
30058         1877
30083         1858
30034         1590
30038         1028
              ... 
35340            1
30354-2473       1
31034            1
31801            1
30344-6733       1
Name: ZIP, Length: 572, dtype: int64

In [89]:
prosnastate = pros[pros['State'].notna()]

In [146]:
prosnadob = pros[pros['DOB'].notna()]

In [163]:
DOB = prosnadob.DOB
age = []

In [164]:
for item in DOB:
    time = pd.Timestamp.now() - pd.Timestamp(item)
    years = time.days/365.24
    age.append(years)

In [149]:
import math

In [179]:
age = pd.Series(age)
whole_age = []
for item in age:
    whole_age.append(math.trunc(item))
whole_age = pd.Series(whole_age)

In [180]:
ranges = {'<18' : 0, '18-25' : 0, '26-35' : 0, '36-45' : 0, '46-55' : 0, '56-65' : 0, '>65' : 0}
for item in whole_age:
    if item < 18:
        ranges['<18'] = ranges['<18'] + 1
    elif item < 26:
        ranges['18-25'] = ranges['18-25'] + 1
    elif item < 36:
        ranges['26-35'] = ranges['26-35'] + 1
    elif item < 46:
        ranges['36-45'] = ranges['36-45'] + 1
    elif item < 56:
        ranges['46-55'] = ranges['46-55'] + 1
    elif item < 66:
        ranges['56-65'] = ranges['56-65'] + 1
    else:
        ranges['>65'] = ranges['>65'] + 1

In [181]:
ranges_list = []
for count in ranges.values():
    ranges_list.append(count)

In [192]:
age_count = ranges_list[0] + ranges_list[1] + ranges_list[2] + ranges_list[3] + ranges_list[4] + ranges_list[5] + ranges_list[6]

for item in ranges_list:
    print((item/age_count) * 100)

0.05145356315924878
14.078266586627791
38.83028899751308
25.563845296286768
12.042992310550838
7.420747220078323
2.0124060257839522
